# Chains

In this notebook we'll learn about creating simple chains in [LangChain](https://python.langchain.com/en/latest/index.html#). Chains are simply a sequence of components, executed in a particular order.

List of main relevant tutorials/blog posts

* [Getting started with chains (Langchain)](https://python.langchain.com/en/latest/modules/chains/getting_started.html)
* [LangChain for LLM Application Development (DeepLearning.AI)](https://learn.deeplearning.ai/langchain/)
* [Briggs and Ingham (2023)](https://www.pinecone.io/learn/langchain/). LangChain AI Handbook.

**TODO:**
* Improve method for counting tokens  so it can be used with complex chains

# 1 - Introduction

## 1.1 - Why do we need chains?

Chains enable us to merge various components into a unified application. For instance, we can form a chain that incorporates user input, applies a [`PromptTemplate`](https://python.langchain.com/en/latest/modules/prompts/prompt_templates.html) to format it, and subsequently transfers the formatted response to a large language model (LLM). By combining multiple chains or integrating them with other components, we can construct even more intricate chains.

The simplest of these chains is the [`LLMChain`](https://python.langchain.com/en/latest/modules/chains/generic/llm_chain.html). It works by 
1. Taking a user's input.
2. Passing in to the first element in the chain (i.e., a [`PromptTemplate`](https://python.langchain.com/en/latest/modules/prompts/prompt_templates.html)) to format the input in order to generate a particular prompt.
3. The formatted prompt is then passed to the next (and final) element in the chain (i.e., a LLM).

We'll start by "creating" our LLM, which is going to be an instance of GPT 3.5

In [15]:
import openai
import os
from langchain.llms import AzureOpenAI

openai.api_type = "azure"
openai.api_base = "https://gpt3tests.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = os.environ["OPENAI_API_KEY"]

engine = "Davinci003"
max_tokens = 1000

llm = AzureOpenAI(deployment_name=engine)
llm.openai_api_key = openai.api_key
llm.openai_api_base = openai.api_base 
llm.max_tokens = max_tokens

We can now create a very simple chain that will take user input, format the prompt with it, and then send it to the LLM.

In [16]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Create the prompt template
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}? (only return one example. Nothing else)",
)

# Create the simple LLMChain
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(llm_chain.run("colorful socks"))



Cheerful Socks Co.


An extra utility we can use is the following function, which will tell us how many tokens we are using in each call. This is a good practice for when we use more complex tools that might make several calls to the API (i.e., agents). It is useful to avoid unsuspected expenditures.

In [17]:
from langchain.callbacks import get_openai_callback

def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

In [18]:
print(count_tokens(llm_chain, "colorful socks"))

Spent a total of 29 tokens


Socktastic.


If there are multiple variables, you can input them all at once using a dictionary:

In [20]:
prompt = PromptTemplate(
    input_variables=["company", "product"],
    template="What is a good name for {company} that makes {product}?",
)
llm_chain = LLMChain(llm=llm, prompt=prompt)
query = {
    'company': "ABC Startup",
    'product': "colorful socks"
    }
print(count_tokens(llm_chain, query))

Spent a total of 20 tokens


Happy Socksy.


## 1.2 - Different ways of calling chains

All classes that inherit from `Chain` offer a few ways of running chain logic. 

### 1.2.1 - `__call__`

By default, `__call__` returns both the input and output key values. 

In [21]:
prompt_template = "Tell me a {adjective} joke"
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)

llm_chain(inputs={"adjective":"corny"})

{'adjective': 'corny',
 'text': '\n\nQ: Why did the scarecrow win the Nobel Prize?\nA: Because he was outstanding in his field.'}

You can configure it to only return output key values by setting `return_only_outputs` to `True`.

In [ ]:
llm_chain("corny", return_only_outputs=True)

### 1.2.2 - `run()`

If the `Chain` only outputs one thing (i.e., only has one element in its `output_keys`), you can use `run` method. Note that `run` outputs a string instead of a dictionary.

In [22]:
# llm_chain only has one output key, so we can use run
llm_chain.output_keys

['text']

In [23]:
llm_chain.run({"adjective":"corny"})

'\n\nQ: Why did the scarecrow win the Nobel Prize? \nA: Because he was outstanding in his field.'

## 1.3 - Add memory to chains

`Chain` supports taking a `BaseMemory` object as its `memory` argument, allowing `Chain` object to persist data across multiple calls. In other words, it makes `Chain` a stateful object.

In [25]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

conversation = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

print(conversation.run("Answer briefly. What are the first 3 colors of a rainbow?"))
# -> The first three colors of a rainbow are red, orange, and yellow.
print(conversation.run("And the next 4?"))
# -> The next four colors of a rainbow are green, blue, indigo, and violet.

 The first three colors of a rainbow are red, orange, and yellow.
 The next four colors of a rainbow are green, blue, indigo, and violet.


Essentially, `BaseMemory` defines an interface of how [langchain](https://python.langchain.com/en/latest/index.html) stores memory. It allows reading of stored data through `load_memory_variables()` method and storing new data through save_context method.

## 1.4 - Debug chains

It can be hard to debug a `Chain` object solely from its output as most `Chain` objects involve a fair amount of input prompt preprocessing and LLM output post-processing. Setting `verbose` to `True` will print out some internal states of the `Chain` object while it is being ran.

In [26]:
conversation = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory(),
    verbose=True
)
conversation.run("What is ChatGPT?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What is ChatGPT?
AI:

> Finished chain.


' ChatGPT is a natural language processing system developed by OpenAI. It is a conversational AI that can generate human-like responses to questions and conversations. It utilizes a large-scale language model based on the GPT-3 architecture to generate natural language responses.'

# 2 - Generic chains

## 2.1 - Transformation chain

Say we have had experience of getting dirty input texts. Specifically, as we know, llms charge us by the number of tokens we use and we are not happy to pay extra when the input has extra characters.

First, we will build a custom transform function to clean the spacing of our texts. We will then use this function to build a chain where we input our text and we expect a clean text as output.

In [27]:
import re

def transform_func(inputs: dict) -> dict:
    text = inputs["text"]
    
    # replace multiple new lines and multiple spaces with a single one
    text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', text)
    text = re.sub(r'[ \t]+', ' ', text)

    return {"output_text": text}

In [28]:
from langchain.chains import TransformChain

clean_extra_spaces_chain = TransformChain(input_variables=["text"], output_variables=["output_text"], transform=transform_func)
clean_extra_spaces_chain.run('A random text  with   some irregular spacing.\n\n\n     Another one   here as well.')

'A random text with some irregular spacing.\n Another one here as well.'

As you can see this auxiliary chain does not require an llm to preprocess the data. It is usually combined with other chains with the following sequential approach.

## 2.2 - Sequential chains

The idea behind sequential chains is to combine multiple chains where output of the one chain is the input of the next chain. There are two types of sequential chains:
* `SimpleSequentialChain`: Single input/output
* `SequentialChain`: multiple inputs/outputs

### 2.2.1 - SimpleSequentialChain

In [29]:
from langchain.chains import SimpleSequentialChain
from langchain.prompts import ChatPromptTemplate

# First chain
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product} (only return one example. Nothing else)?"
)
first_chain = LLMChain(llm=llm, prompt=first_prompt)

# Second chain
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following company: {company_name}"
)
second_chain = LLMChain(llm=llm, prompt=second_prompt)

In [31]:
simple_chain = SimpleSequentialChain(chains=[first_chain, second_chain], verbose=True)
print(count_tokens(simple_chain, "colorful socks"))



> Entering new SimpleSequentialChain chain...


Vibrant Socks


A fun and stylish company offering a wide range of colorful and unique socks to brighten up any outfit.

> Finished chain.
Spent a total of 74 tokens


A fun and stylish company offering a wide range of colorful and unique socks to brighten up any outfit.


### 2.2.2 - SequentialChain

The `SimpleSequentialChain` works well when we have a single input and a single output. But, what happens when we have multiple inputs and outputs?

As an example, we are going to create a fake review, translate it to English, summarize it and answer it in the original language (i.e., Spanish)

<table>
    <tr>
        <td><img src="images_1/sequential_chain_example.png" width="700"/></td>
    </tr>
</table>

#### Step 0: Create fake Spanish review

In [38]:
base_prompt = ChatPromptTemplate.from_template(
    "Create a 50 word fake positive review about a product of a company called {company} in Spanish:"
)
base_chain = LLMChain(llm=llm, prompt=base_prompt, output_key="review")
# print(count_tokens(base_chain, "colorful socks")) # Test the chain

#### Step 1: Translate review to English

In [39]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{review}"
)
# chain 1: input= Review and output= English_Review
first_chain = LLMChain(llm=llm, prompt=first_prompt, output_key="english_review")

#### Step 2: Summarize English review

In [40]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{english_review}"
)
# chain 2: input= English_Review and output= summary
second_chain = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")

#### Step 3: Identify language of review

In [41]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{review}"
)
# chain 3: input= Review and output= language
third_chain = LLMChain(llm=llm, prompt=third_prompt, output_key="language")

#### Step 4: Write followup response

In [42]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
fourth_chain = LLMChain(llm=llm, prompt=fourth_prompt, output_key="followup_message")

#### Create sequential chain

In [43]:
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[base_chain, first_chain, second_chain, third_chain, fourth_chain],
    input_variables=["company"],
    output_variables=["review", "english_review", "summary", "followup_message"],
    verbose=True
)

company = "colorful socks"
overall_chain(company)



> Entering new SequentialChain chain...

> Finished chain.


{'company': 'colorful socks',
 'review': '\n\n"Los calcetines Colorful Socks son los mejores! Son cómodos, duraderos y siempre hay una variedad de colores y estilos para elegir. El servicio al cliente es excelente, siempre responden rápidamente y ofrecen un envío rápido. Me encantan los calcetines Colorful Socks y los recomiendo a todos mis amigos."',
 'english_review': '\n\n"Colorful Socks socks are the best! They are comfortable, durable and there is always a variety of colors and styles to choose from. Customer service is excellent, they always respond quickly and offer fast shipping. I love Colorful Socks socks and I recommend them to all my friends."',
 'summary': '\n\nColorful Socks socks are excellent in quality, comfort, variety, customer service, and shipping.',
 'followup_message': '\n\n¡Es genial oír que Colorful Socks ofrece una gran calidad, comodidad, variedad, servicio al cliente y envío! Estoy impresionado con la excelente experiencia que ofrecen. ¡Gracias por compartir

## 2.3 - Router chain

We basically generate a specific chain backed by a LLM that redirects to the appropriate sub-chain by reading the question and the prompt template descriptions

<table>
    <tr>
        <td><img src="images_1/router_chain_example.png" width="700"/></td>
    </tr>
</table>

In [44]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

In [45]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [46]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [47]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [48]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [49]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

**Note:** we do not need to manually implement this template every time, we can import it using:

`from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE`

In [58]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [51]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [ ]:
chain.run("what is 2 + 2")

In [55]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"\n\nAnswer: Black body radiation is the electromagnetic radiation emitted by an idealized black body, which is a body that absorbs all electromagnetic radiation that hits it. It is a type of thermal radiation, which means it is produced by the thermal motion of the particles in the body. The radiation has a characteristic spectrum that is dependent on the body's temperature, and in the case of a black body, the spectrum is continuous across all wavelengths."

In [ ]:
chain.run("Why does every cell in our body contain DNA?")